In [34]:
import numpy as np
from pyriemann.utils.base import logm, expm
from scipy.linalg import sqrtm, eigh

# Classe CalculCovariance
class CalculCovariance:
    def __init__(self, p_points, stride):
        self.p_points = p_points
        self.stride = stride
    
    def calculer_matrices_covariance(self, donnees):
        k, n = donnees.shape
        matrices_covariance = []
        
        for start_index in range(0, n - self.p_points + 1, self.stride):
            end_index = start_index + self.p_points
            segment = donnees[:, start_index:end_index]
            covariance_matrix = np.cov(segment) + np.eye(k) * 1e-6  # Régularisation ajoutée
            matrices_covariance.append(covariance_matrix)
        
        return matrices_covariance

In [36]:
# Classe Trajectoire
class Trajectoire:
    def __init__(self, *covariance_matrices):
        self.covariance_matrices = list(covariance_matrices)
        self.trajectoire = None
    
    def ajouter_covariance(self, nouvelle_covariance):
        self.covariance_matrices.append(nouvelle_covariance)
        self.trajectoire = None
    
    def calculer_trajectoire(self):
        self.trajectoire = sum(self.covariance_matrices)
    
    def obtenir_covariances(self):
        return self.covariance_matrices
    
    def obtenir_trajectoire(self):
        if self.trajectoire is None:
            self.calculer_trajectoire()
        return self.trajectoire
    
    def __str__(self):
        return (f'Trajectoire avec {len(self.covariance_matrices)} matrices de covariance\n'
                f'Trajectoire actuelle:\n{self.trajectoire}')

In [77]:
# Classe GestionnaireTrajectoires
class GestionnaireTrajectoires:
    def __init__(self, max_matrices_par_trajectoire, *initial_covariances):
        self.trajectoires = []
        self.max_matrices_par_trajectoire = max_matrices_par_trajectoire
        if initial_covariances:
            traj = Trajectoire(*initial_covariances[:max_matrices_par_trajectoire])
            self.trajectoires.append(traj)
            remaining_covariances = initial_covariances[max_matrices_par_trajectoire:]
            while remaining_covariances:
                traj = Trajectoire(*remaining_covariances[:max_matrices_par_trajectoire])
                self.trajectoires.append(traj)
                remaining_covariances = remaining_covariances[max_matrices_par_trajectoire:]
    
    def ajouter_covariance(self, nouvelles_covariance):
        for nouvelle_covariance in nouvelles_covariance:
            if not self.trajectoires or len(self.trajectoires[-1].obtenir_covariances()) >= self.max_matrices_par_trajectoire:
                self.trajectoires.append(Trajectoire(nouvelle_covariance))
            else:
                self.trajectoires[-1].ajouter_covariance(nouvelle_covariance)
    
    def obtenir_trajectoires(self):
        return [traj.obtenir_trajectoire() for traj in self.trajectoires]
    
    def tangent_space_projection(self, A, B):
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_A = np.dot(np.dot(sqrt_inv_B, A), sqrt_inv_B)
        log_transformed_A = logm(transformed_A)
        return log_transformed_A
    
    def inverse_tangent_space_projection(self, ProjB_A, B):
        sqrt_B = sqrtm(B)
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        transformed_ProjB_A = np.dot(np.dot(sqrt_inv_B, ProjB_A), sqrt_inv_B)
        exp_transformed_ProjB_A = expm(transformed_ProjB_A)
        original_A = np.dot(np.dot(sqrt_B, exp_transformed_ProjB_A), sqrt_B)
        return original_A
    
    def transport_vector(self, Delta, A, B):
        inv_B = np.linalg.inv(B)
        A_invB = np.dot(A, inv_B)
        E = sqrtm(A_invB)
        transported_Delta = np.dot(np.dot(E, Delta), E.T)
        return transported_Delta

    def projeter_et_transporter(self):
        if len(self.trajectoires) == 0:
            return

        for traj in self.trajectoires:
            covariances = traj.obtenir_covariances()
            print(covariances[0])
            projected_covariances = []

            for i in range(len(covariances) - 1):
                proj = self.tangent_space_projection(covariances[i], covariances[i + 1])
                if projected_covariances:
                    for j in range(len(projected_covariances)):
                        projected_covariances[j] = self.transport_vector(projected_covariances[j], covariances[i], covariances[i + 1])
                projected_covariances.append(proj)

            # Ajouter la dernière covariance sans projection car elle est déjà au bon endroit
            projected_covariances.append(covariances[-1])

            # Remplacer les covariances dans la trajectoire par les covariances projetées et transportées
            traj.covariance_matrices = projected_covariances

            # Calculer la trajectoire après projection et transport
            traj.calculer_trajectoire()
    
    def __str__(self):
        result = ''
        for i, traj in enumerate(self.trajectoires):
            result += f'Trajectoire {i+1}:\n{traj}\n'
        return result

In [78]:
import numpy as np
import mne

class GenerateurDonnees:
    def __init__(self, channels=3, sfreq=256.0, duration=0.1, change=5.0, A1=2.0e-6, A2=0.0e-6, sd=2.0e-6):
        self.channels = channels
        self.sfreq = sfreq
        self.duration = duration
        self.change = change
        self.frequencies = 10.0 + 2.0 * np.random.rand(channels)
        self.phases = 2.0 * np.pi * np.random.rand(channels)
        self.A1 = A1
        self.A2 = A2
        self.sd = sd
        self.info = mne.create_info([f'EEG{n:03}' for n in range(1, channels + 1)],
                                    ch_types=['eeg'] * channels, sfreq=sfreq)

    def generer_donnees(self):
        samples = int(self.sfreq * self.duration)
        t = np.linspace(0, self.duration, samples, endpoint=False)
        A = self.A1 + (t > self.change).astype(float) * (self.A2 - self.A1)
        data = A * np.cos(np.outer(self.frequencies, t) + np.outer(self.phases, np.ones(samples)))
        data += np.random.normal(0, self.sd, size=(self.channels, samples))
        raw = mne.io.RawArray(data, self.info)
        return raw

    def exporter_donnees(self, filename='dummy.edf'):
        raw = self.generer_donnees()
        mne.export.export_raw(filename, raw, fmt='edf', overwrite=True)







In [13]:
# Usage example
generateur = GenerateurDonnees()
donnees = generateur.generer_donnees()

Creating RawArray with float64 data, n_channels=3, n_times=2560
    Range : 0 ... 2559 =      0.000 ...     9.996 secs
Ready.


In [88]:
# Fonction de test
def test_gestionnaire_trajectoires(donnees, p_points, stride, nb_cov_matrix_per_traj, file):
    file.write("----- Test GestionnaireTrajectoires -----\n")
    file.write(f"Données:\n{donnees}\n")
    file.write(f"p_points: {p_points}\n")
    file.write(f"stride: {stride}\n")
    file.write(f"Nombre max de matrices par trajectoire: {nb_cov_matrix_per_traj}\n")

    calcul_cov = CalculCovariance(p_points, stride)
    matrices_covariance = calcul_cov.calculer_matrices_covariance(donnees)
    gestionnaire = GestionnaireTrajectoires(nb_cov_matrix_per_traj, *matrices_covariance)
    gestionnaire.projeter_et_transporter()
    file.write("État initial des trajectoires:\n")
    file.write(str(gestionnaire) + "\n")

    #calcul_cov1 = CalculCovariance(p_points, stride)
    #matrices_covariance1 = calcul_cov1.calculer_matrices_covariance(donnees1)
    #nouvelle_covariance = np.cov(donnees[:, -p_points:]) + np.eye(donnees.shape[0]) * 1e-6
    #gestionnaire.ajouter_covariance(matrices_covariance1)
    #gestionnaire.projeter_et_transporter()
    
    file.write("État des trajectoires après ajout de la nouvelle covariance:\n")
    file.write(str(gestionnaire) + "\n")
    return gestionnaire

def test_gestionnaire_trajectoires_apres_ajout(gestionnaire, donnees, p_points, stride, nb_cov_matrix_per_traj, file):
    file.write("----- Test GestionnaireTrajectoires -----\n")
    file.write(f"Données:\n{donnees}\n")
    file.write(f"p_points: {p_points}\n")
    file.write(f"stride: {stride}\n")
    file.write(f"Nombre max de matrices par trajectoire: {nb_cov_matrix_per_traj}\n")
    calcul_cov = CalculCovariance(p_points, stride)
    matrices_covariance = calcul_cov.calculer_matrices_covariance(donnees)
    #nouvelle_covariance = np.cov(donnees[:, -p_points:]) + np.eye(donnees.shape[0]) * 1e-6
    gestionnaire.ajouter_covariance(matrices_covariance)
    gestionnaire.projeter_et_transporter()


In [89]:
if __name__ == "__main__":

    generateur = GenerateurDonnees()
    donnees = generateur.generer_donnees()
    donnees = donnees.get_data()

    generateur1 = GenerateurDonnees()
    donnees1 = generateur.generer_donnees()
    donnees1 = donnees1.get_data()

    scenarios = [
        (donnees, 3, 1, 2),
        (donnees, 3, 2, 3),
        (donnees, 3, 3, 4),
    ]

    scenarios1 = [
        (donnees1, 3, 1, 2),
        (donnees1, 3, 2, 3),
        (donnees1, 3, 3, 4),
    ]


    with open("resultats_tests.txt", "w") as file:
        for donnees, p_points, stride, nb_cov_matrix_per_traj in scenarios:
            gestionnaire=test_gestionnaire_trajectoires(donnees, p_points, stride, nb_cov_matrix_per_traj, file)

    #with open("resultats_tests_apres_ajout.txt", "w") as file:
        #for donnees1, p_points, stride, nb_cov_matrix_per_traj in scenarios1:
            #test_gestionnaire_trajectoires_apres_ajout(gestionnaire,donnees1, p_points, stride, nb_cov_matrix_per_traj, file)

Creating RawArray with float64 data, n_channels=3, n_times=25
    Range : 0 ... 24 =      0.000 ...     0.094 secs
Ready.
Creating RawArray with float64 data, n_channels=3, n_times=25
    Range : 0 ... 24 =      0.000 ...     0.094 secs
Ready.
[[ 1.00000068e-06 -1.07196406e-12 -7.05727206e-13]
 [-1.07196406e-12  1.00000428e-06  5.33990361e-12]
 [-7.05727206e-13  5.33990361e-12  1.00000763e-06]]
[[ 1.00000240e-06 -1.60294667e-12 -3.96234635e-12]
 [-1.60294667e-12  1.00000511e-06  3.91698704e-12]
 [-3.96234635e-12  3.91698704e-12  1.00000694e-06]]
[[ 1.00000497e-06  4.40099216e-12  5.79790434e-13]
 [ 4.40099216e-12  1.00000498e-06 -1.41667039e-12]
 [ 5.79790434e-13 -1.41667039e-12  1.00000353e-06]]
[[1.00000762e-06 2.38217003e-12 1.72923645e-12]
 [2.38217003e-12 1.00000133e-06 9.78452312e-13]
 [1.72923645e-12 9.78452312e-13 1.00000072e-06]]
[[ 1.00000493e-06  4.38776289e-12 -1.15271594e-12]
 [ 4.38776289e-12  1.00000393e-06 -1.03586984e-12]
 [-1.15271594e-12 -1.03586984e-12  1.00000027e-

: 